In [1]:
import sys, os
sys.path.append('./modules')
from Monitoring_Tools import *
from OpenStack_Tools  import *

In [2]:
platform=os.getenv('OS_PLATFORM', 'demo8')
display_platform(platform)
inventory = read_inventory("$HOME/env/{}_hosts.ini".format(platform))
conn = connectToCloud(platform)
#inventory

In [3]:
HTML_URL = show_notebook_url(platform, host_ip="10.3.216.210", port=8888)

TypeError: linkto_notebook_url() missing 2 required positional arguments: 'platform' and 'host_ip'

#### Servers status (ping servers in 'ping_check' group)

In [ ]:
display_html_ping_all(inventory)

#### Show VMs (servers)

In [ ]:
displayServerList( conn )

#### Show server port status
Note: display_html_ping_ports_all defaults to group 'ssh_check' and port 22

In [ ]:
display_html_ping_ports_all(inventory)

#### Show Endpoint URLs status (check ports open)

In [ ]:
display_html_endpoint_urls(conn)

In [ ]:
for host in sorted(inventory['ssh_check']):    
    ip = inventory['hosts'][host]['ansible_host']
    user = inventory['hosts'][host]['ansible_user']
    pkey = inventory['hosts'][host]['ssh_key']
    
    stdout, stderr = ssh_command(host, ip, user, pkey, "uptime")
    
    #print("LINE=" + stdout)
    uptime = strip_uptime(stdout)
    print(host + ":" + uptime)

In [ ]:
! [ ! -d history ] && mkdir history

import datetime
import time

#d = datetime.date.today().strftime("%B %d, %Y")
#dt = datetime.datetime.now().strftime("%I:%M%p on %B %d, %Y")
d = datetime.date.today().strftime("%Y-%m-%d")
dt = datetime.datetime.now().strftime("%Y-%m-%d_%H:%M")
print(d)
print(dt)

In [ ]:
for host in sorted(inventory['df_check']):    
    ip = inventory['hosts'][host]['ansible_host']
    user = inventory['hosts'][host]['ansible_user']
    pkey = inventory['hosts'][host]['ssh_key']
    df_check = inventory['hosts'][host]['df_check']
    
    # write to history subdir (~/notebooks/cron for cron jobs)
    history_file='history/df_history_' + platform + '_' + host + '.txt'
    history_fd = open(history_file, 'a')
    
    full_df_cmd="hostname; df 2>&1"
    df_op, stderr = ssh_command(host, ip, user, pkey, full_df_cmd)    
    history_fd.write('DATE:' + dt + '\n' + df_op)
    history_fd.close()
    
    df_cmd="df " + df_check.replace(",", " ") + "| grep -v ^Filesystem"
    df_op, stderr = ssh_command(host, ip, user, pkey, df_cmd)    
    #df_op = stdout.decode('utf8')
    #print("HOST[" + host + "]<" + df_check + ">{" + df_cmd +"}:" + df_op)
    
    df_lines=df_op.split("\n")
    for df_line in df_lines:
        #print("LINE: " + df_line)
        pc_pos = df_line.find("%")
        if pc_pos != -1:
            pc=int(df_line[pc_pos-3:pc_pos])
            partn=df_line[pc_pos+1:]
            print(host + " " + str(pc) + "% " + partn)

# Attempts at embedding matplotlib graphic output
.
This is flawed for nbconvert/e-mail because the e-mail client will not render the image for security reasons

Although graphics will not be included in the e-mail, they are included in the nbconvert generated html


In [ ]:
%matplotlib inline

import matplotlib
import numpy as np
import matplotlib.pyplot as plt

x = np.linspace(0, 3*np.pi, 500)
plt.plot(x, np.sin(x**2))
plt.title('A simple chirp');

In [ ]:
from ipywidgets import interactive
from IPython.display import Audio, display
import numpy as np

#### Similarly other graphics generated through SVG, CSS are displayed in the nbconvert generated html

# Attempts at creating a pie chart using CSS.
This is flawed for nbconvert/e-mail because the e-mail client will not render the CSS for security reasons


In [ ]:
%%html

<style>
     .pieContainer {
          height: 260px;
          position: relative;
     }
     .pieBackground {
          background-color: lightgrey;
          position: relative;
          width: 180px;
          height: 180px;
          -moz-border-radius: 90px;
          -webkit-border-radius: 90px;
          -o-border-radius: 90px;
          border-radius: 90px;
          -moz-box-shadow: -1px 1px 3px #000;
          -webkit-box-shadow: -1px 1px 3px #000;
          -o-box-shadow: -1px 1px 3px #000;
          box-shadow: -1px 1px 3px #000;
     } 
     .pie {
          position: absolute;
          width: 180px;
          height: 180px;
          -moz-border-radius: 90px;
          -webkit-border-radius: 90px;
          -o-border-radius: 90px;
          border-radius: 90px;
          clip: rect(0px, 90px, 180px, 0px);
     }
     .hold {
          position: absolute;
          width: 180px;
          height: 180px;
          -moz-border-radius: 90px;
          -webkit-border-radius: 90px;
          -o-border-radius: 90px;
          border-radius: 90px;
          clip: rect(0px, 180px, 180px, 90px);
     }
     #pieSliceX10 .pie {
          background-color: #1b458b;
          -webkit-transform:rotate(10deg);
          -moz-transform:rotate(10deg);
          -o-transform:rotate(10deg);
          transform:rotate(10deg);
     }
</style>

<div class="pieContainer">
     <h3>Disk usage</h3><p/>
     <div class="pieBackground"> <div id="pieSliceX10" class="hold"><div class="pie"></div></div></div>
    
</div>

In [ ]:
%%html

<!-- ADAPTED from https://codepen.io/AtomicNoggin/pen/fEish -->
    
<div class="pieContainer2">
  <div class="pie2" data-start="0" data-value="30"></div>
  <div class="pie2 highlight" data-start="30" data-value="30"></div>
  <div class="pie2" data-start="60" data-value="40"></div>
  <div class="pie2 big" data-start="100" data-value="260"></div>
</div>

<style>
     .pieContainer2 {
          height: 260px;
          position: relative;
     }

.pie2 {
		position:absolute;
		width:100px;
		height:200px;
		overflow:hidden;
		left:150px;
		-moz-transform-origin:left center;
		-ms-transform-origin:left center;
		-o-transform-origin:left center;
		-webkit-transform-origin:left center;
		transform-origin:left center;
	}
/*
  unless the piece represents more than 50% of the whole chart.
  then make it a square, and ensure the transform origin is
  back in the center.

  NOTE: since this is only ever a single piece, you could
  move this to a piece specific rule and remove the extra class
*/
	.pie2.big {
		width:200px;
		height:200px;
		left:50px;
		-moz-transform-origin:center center;
		-ms-transform-origin:center center;
		-o-transform-origin:center center;
		-webkit-transform-origin:center center;
		transform-origin:center center;
	}
/*
  this is the actual visible part of the pie. 
  Give it the same dimensions as the regular piece.
  Use border radius make it a half circle.
  move transform origin to the middle of the right side.
  Push it out to the left of the containing box.
*/
	.pie2:BEFORE {
		content:"";
		position:absolute;
		width:100px;
		height:200px;
		left:-100px;
		border-radius:100px 0 0 100px;
		-moz-transform-origin:right center;
		-ms-transform-origin:right center;
		-o-transform-origin:right center;
		-webkit-transform-origin:right center;
		transform-origin:right center;
		
	}
 /* if it's part of a big piece, bring it back into the square */
	.pie2.big:BEFORE {
		left:0px;
	}
/* 
  big pieces will also need a second semicircle, pointed in the
  opposite direction to hide the first part behind.
*/
	.pie2.big:AFTER {
		content:"";
		position:absolute;
		width:100px;
		height:200px;
		left:100px;
		border-radius:0 100px 100px 0;
	}
/*
  add colour to each piece.
*/
	.pie2:nth-of-type(1):BEFORE,
	.pie2:nth-of-type(1):AFTER {
		background-color:blue;	
	}
	.pie2:nth-of-type(2):AFTER,
	.pie2:nth-of-type(2):BEFORE {
		background-color:green;	
	}
	.pie2:nth-of-type(3):AFTER,
	.pie2:nth-of-type(3):BEFORE {
		background-color:red;	
	}
	.pie2:nth-of-type(4):AFTER,
	.pie2:nth-of-type(4):BEFORE {
		background-color:orange;	
	}
/*
  now rotate each piece based on their cumulative starting
  position
*/
	.pie2[data-start="30"] {
		-moz-transform: rotate(30deg); /* Firefox */
		-ms-transform: rotate(30deg); /* IE */
		-webkit-transform: rotate(30deg); /* Safari and Chrome */
		-o-transform: rotate(30deg); /* Opera */
		transform:rotate(30deg);
	}
	.pie2[data-start="60"] {
		-moz-transform: rotate(60deg); /* Firefox */
		-ms-transform: rotate(60deg); /* IE */
		-webkit-transform: rotate(60deg); /* Safari and Chrome */
		-o-transform: rotate(60deg); /* Opera */
		transform:rotate(60deg);
	}
	.pie2[data-start="100"] {
		-moz-transform: rotate(100deg); /* Firefox */
		-ms-transform: rotate(100deg); /* IE */
		-webkit-transform: rotate(100deg); /* Safari and Chrome */
		-o-transform: rotate(100deg); /* Opera */
		transform:rotate(100deg);
	}
/*
  and rotate the amount of the pie that's showing.

  NOTE: add an extra degree to all but the final piece, 
  to fill in unsightly gaps.
*/
	.pie2[data-value="30"]:BEFORE {
		-moz-transform: rotate(31deg); /* Firefox */
		-ms-transform: rotate(31deg); /* IE */
		-webkit-transform: rotate(31deg); /* Safari and Chrome */
		-o-transform: rotate(31deg); /* Opera */
		transform:rotate(31deg);
	}
	.pie2[data-value="40"]:BEFORE {
		-moz-transform: rotate(41deg); /* Firefox */
		-ms-transform: rotate(41deg); /* IE */
		-webkit-transform: rotate(41deg); /* Safari and Chrome */
		-o-transform: rotate(41deg); /* Opera */
		transform:rotate(41deg);
	}
	.pie2[data-value="260"]:BEFORE {
		-moz-transform: rotate(260deg); /* Firefox */
		-ms-transform: rotate(260deg); /* IE */
		-webkit-transform: rotate(260deg); /* Safari and Chrome */
		-o-transform: rotate(260deg); /* Opera */
		transform:rotate(260deg);
	}

</style> 

### TODO:
- Create status at top of page (without Javascript - so visible in email?)
- df processing to show trends (as html table, as pie chart)
- use of openstack clientmanager for equivalent to "openstack service list" status

In [ ]:
# TODO: 
# equivalent of openstack service list

# See: how service list is implemented:
#    https://github.com/openstack/python-openstackclient/blob/master/openstackclient/compute/v2/service.py
# (compute_client = client_manager.compute)
#
# See also: how to create/use a clientmanager:
#    https://github.com/openstack/python-openstackclient/blob/master/examples/osc-lib.py

#from openstackclient.common import clientmanager
#from openstackclient.common import utils

# Attempt at hiding code using javascript.

This is flawed for nbconvert/e-mail because the javascript will only be executed (it at all) when the page is actually rendered.

In the case of an html e-mail the javascript won't be rendered so we don't get the platform list we see below.


In [ ]:
from IPython.display import display
from IPython.display import HTML
import IPython.core.display as di

# This line will hide code by default when the notebook is exported as HTML
di.display_html('<script>jQuery(function() {if (jQuery("body.notebook_app").length == 0) { jQuery(".input_area").toggle(); jQuery(".prompt").toggle();}});</script>', raw=True)

# This line will add a button to toggle visibility of code blocks, for use with the HTML export version
di.display_html('''<button onclick="jQuery('.input_area').toggle(); jQuery('.prompt').toggle();">Toggle code</button>''', raw=True)

# Attempt at writing to a div using javascript.

This is flawed for nbconvert/e-mail because the javascript will only be executed (it at all) when the page is actually rendered.

In the case of an html e-mail the javascript won't be rendered so we don't get the platform list we see below.


In [ ]:
import IPython

IPython.display.HTML('<h1>Platform status(es)</h1><div id="status"></div>')

In [ ]:
from IPython.display import Javascript

js = """
    $('#status').html('');

"""
js = Javascript(js)
display(js)

for platform in ['demo8', 'poc1', 'poc2', 'nfv5']:
    #platform=os.getenv('OS_PLATFORM', 'demo8')
    #print(platform)
    js = """
    console.log('plaform=""" + platform + """');
    $('#status').html($('#status').html() + '<h2>Platform:""" + platform + """</h2>');

"""
    js = Javascript(js)
    display(js)